In [1]:
from termcolor import colored
import re
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

In [2]:
# Load tokenizer and model from local directory
model_dir = r"E:\NER_model"  # Replace with the actual path
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForTokenClassification.from_pretrained(model_dir)

In [3]:
# Create NER pipeline
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer)

In [16]:
nationalities_lst = ['فلسطين','اسرائيل','امريكي', 'أفغاني', 'ألباني', 'جزائري', 'أرجنتيني', 'أرميني', 'أسترالي', 'نمساوي', 'أذربيجاني', 'بحريني', 'بنغلاديشي', 'بيلاروسي', 'بلجيكي', 'بوليفي', 'بوسني', 'برازيلي', 'بلغاري', 'كندي', 'تشيلي', 'صيني', 'كولومبي', 'كوستاريكي', 'كرواتي', 'كوبي', 'تشيكي', 'دنماركي', 'دومينيكاني', 'هولندي', 'إكوادوري', 'مصري', 'سلفادوري', 'إماراتي', 'استوني', 'إثيوبي', 'فنلندي', 'فرنسي', 'جورجي', 'ألماني', 'غاني', 'يوناني', 'غواتيمالي', 'هايتي', 'هندوراسي', 'هنغاري', 'أيسلندي', 'هندي', 'إندونيسي', 'إيراني', 'عراقي', 'أيرلندي', 'إسرائيلي', 'إيطالي', 'جامايكي', 'ياباني', 'أردني', 'كازاخستاني', 'كيني', 'كويتي', 'قيرغيزستاني', 'لاتفي', 'لبناني', 'ليبي', 'ليتواني', 'لوكسمبورغي', 'مقدوني', 'ماليزي', 'مالطي', 'مكسيكي', 'مولدوفي', 'منغولي', 'مونتينيغرين', 'مغربي', 'ناميبي', 'نيبالي', 'نيوزيلندي', 'نيكاراغواي', 'نيجيري', 'نرويجي', 'عماني', 'باكستاني', 'بنمي', 'باراغواي', 'بيروفي', 'فلبيني', 'بولندي', 'برتغالي', 'قطري', 'روماني', 'روسي', 'سعودي', 'صربي', 'سنغافوري', 'سلوفاكي', 'سلوفيني', 'صومالي', 'جنوب أفريقي', 'اوكراني', 'اميركي']

In [5]:
def return_named_entities(text):
    # Perform NER on the text
    results = ner_pipeline(text)

    # Store named entities in a list of dictionaries
    named_entities = []

    for result in results:
        entity_info = {
            "Entity": result['word'],
            "Label": result['entity'],
            "Score": result['score']
        }
        named_entities.append(entity_info)
        
    return named_entities

In [6]:
def return_combined_results(named_entities):
    combined_results = []
    current_entity = {"word": "", "label": "", "score": 0.0}

    for result in named_entities:
        entity_word = result["Entity"]
        label = result["Label"]
        score = result["Score"]

        # Check if the token starts with "##" and append it to the previous word
        if entity_word.startswith("##"):
            current_entity["word"] += entity_word[2:]
        else:
            # Non-continuation token, add the current entity to the results
            if current_entity["word"]:
                combined_results.append(current_entity.copy())
                current_entity = {"word": "", "label": "", "score": 0.0}

            current_entity["word"] = entity_word
            current_entity["label"] = label
            current_entity["score"] = score

    # Add the last entity to the results if any
    if current_entity["word"]:
        combined_results.append(current_entity.copy())
        
    return combined_results

In [7]:
def return_matched_nationalities(text, nationalities_lst):

    matched_nationalities = set()

    # Replace "أ" with "ا"
    text = text.replace("أ", "ا")

    # Replace "إ" with "ا"
    text = text.replace("إ", "ا")

    # Define the part of the word you want to search for
    for element in nationalities_lst:
        search_part = element

        # Create a regex pattern to find the search part
        pattern = re.compile(r"\b\w*" + re.escape(search_part) + r"\w*\b")

        # Find all the matched words in the sentence
        matched_words = re.findall(pattern, text)

        # Check if any words are found
        if matched_words:
            for word in matched_words:
                matched_nationalities.add(word)
        else:
            continue  
            
    return matched_nationalities

In [15]:
def extract_arabic_dates(text):
    # Define a regex pattern for the date format with a single slash
    pattern = re.compile(r'\b(\d{1,2})\\(\d{1,2})\\(\d{4})\b')

    # Find all matches in the text
    matches = re.findall(pattern, text)

    # Format the matches as "١٠\١٠\٢٠٢٣"
    formatted_dates = [f"{day}\\{month}\\{year}" for day, month, year in matches]

    return formatted_dates

In [9]:
def return_persons_names(combined_results):
    # Join and return consecutive words with the label "B-PERSON"
    joined_persons = []
    current_person = []

    for entity in combined_results:
        if entity['label'] == 'B-PERSON' or entity['label'] == 'I-PERSON':
            current_person.append(entity['word'])
        elif current_person:
            joined_persons.append(' '.join(current_person))
            current_person = []

    # Add the last person if any
    if current_person:
        joined_persons.append(' '.join(current_person))

    # Print or return the joined persons
    return joined_persons

In [10]:
def return_key_words(combined_results, dates, matched_nationalities, persons_names):
    # Create an empty set
    key_words = set()

    for result in combined_results:
#         if result["label"] != 'B-PRODUCT' and result["label"] != 'I-PRODUCT' and result["label"] != 'B-ORGANIZATION' and result["label"] != "I-DATE" and result["label"] != "B-DATE" and result["label"] != "I-PERSON" and result["label"] != "B-PERSON":
        if result["label"] == "I-LOCATION" or result["label"] == "B-LOCATION" or result["label"] == "I-ORGANIZATION":
            key_words.add(result["word"])
        else:
            continue

    for date in dates:
        key_words.add(dates)

    for nationality in matched_nationalities:
        key_words.add(nationality)

    for person in persons_names:
        key_words.add(person)
        
    return key_words

In [11]:
# def return_colored_text(text, matched_nationalities, combined_results, dates):
#     # Check if the word is a nationality
#     for result in matched_nationalities:
#         text = text.replace(result, colored(result, on_color='on_green'))

#     for result in combined_results:
#         if result['label'] == "B-PERSON" or result['label'] == "I-PERSON":
#             text = text.replace(result['word'], colored(result['word'], on_color='on_blue'))
#         elif result['label'] == "B-LOCATION" or result['label'] == "I-LOCATION":
#             text = text.replace(result['word'], colored(result['word'], on_color='on_green'))
#         elif result['label'] == "B-ORGANIZATION" or result['label'] == "I-ORGANIZATION":
#             text = text.replace(result['word'], colored(result['word'], on_color='on_red'))
#         elif result['label'] == "B-EVENT":
#             text = text.replace(result['word'], colored(result['word'], on_color='on_yellow'))
#         elif result['label'] == "I-EVENT":
#             text = text.replace(result['word'], colored(result['word'], on_color='on_magenta'))
#         elif result['label'] == "I-LOCATION":
#             text = text.replace(result['word'], colored(result['word'], on_color='on_green'))

#     for date in dates:
#         text = text.replace(i, colored(i, on_color='on_white'))
#     # Print or return the colored sentence
#     return colored_text

In [17]:
def return_results(text):
    named_entities = return_named_entities(text)
    combined_results = return_combined_results(named_entities)
    matched_nationalities = return_matched_nationalities(text, nationalities_lst)
    dates = extract_arabic_dates(text)
    persons_names = return_persons_names(combined_results)
    key_words = return_key_words(combined_results, dates, matched_nationalities, persons_names)
    return key_words

In [18]:
text = "أكد وزير الخارجية السعودي، اليوم السبت، أن الأحداث المأساوية في فلسطين تحتم علينا التحرك العاجل لوقف إطلاق النار، معلناً رفض بلاده بشكل قاطع تهجير الفلسطينيين من أراضيهم. وطالب خلال كلمته في قمة السلام بالقاهرة بفتح فوري لممرات إنسانية آمنة إلى غزة، داعياً المجتمع الدولي إلى إلزام إسرائيل بالتقيد بالقانون الدولي. وقال خلال مشاركته في القمة: «نرفض محاولات التهجير القسري للفلسطينيين، وإننا نعرب عن خيبة أملنا في عجز مجلس الأمن الدولي عن اتخاذ موقف حيال الأزمة الحالية في غزة حتى الآن، ونطالب بفتح فوري لممرات إنسانية آمنة». وعبر وزير الخارجية السعودية عن أمله في أن تسهم هذه القمة في تحرك حاسم للمجتمع الدولي لإيجاد حل لهذه الأزمة، مؤكداً على «تمسك المملكة بالسلام خياراً استراتيجياً عبر الوقوف مع الشعب الفلسطيني لاستعادة حقوقه المشروعة». وقدم وزير الخارجية السعودي الشكر للرئيس المصري عبد الفتاح السيسي، وحكومة مصر، على الجهود المبذولة «لتعزيز التنسيق والتشاور الإقليمي والدولي تجاه ما نشهده من تطورات خطيرة في قطاع غزة». كان الأمير فيصل بن فرحان بن عبد الله وزير الخارجية السعودي، وصل اليوم إلى العاصمة المصرية القاهرة، لترؤس وفد المملكة المشارك في قمة القاهرة للسلام، والمنعقدة بشأن مناقشة الصراع في غزة ومحيطها، ومستقبل القضية الفلسطينية، وذلك نيابةً عن الأمير محمد بن سلمان بن عبد العزيز ولي العهد رئيس مجلس الوزراء كما شارك بوتين. ويشارك في القمة عدد من الدول والمنظمات الدولية لبحث تطورات الأوضاع الراهنة في فلسطين، والعمل على حماية المدنيين في قطاع غزة المُحاصر، والسماح بفتح ممرات آمنة، وإدخال المساعدات الإنسانية والطبية والغذائية"

In [19]:
results = return_results(text)

In [20]:
print(results)

{'السعودي', 'فيصل بن فرحان بن عبد الله', 'الفلسطيني', 'الفلسطينية', 'اسرائيل', 'محمد بن سلمان بن عبد العزيز', 'الفلسطينيين', 'عبد الفتاح السيسي', 'بالقاهرة', 'غزة', 'الامن', 'السعودية', 'القاهرة', 'الدولي', 'بوتين', 'للفلسطينيين', 'فلسطين', 'المصري', 'الوزراء', 'المصرية', 'مصر', 'قطاع'}
